<h3>Integrating Python and MySQL for Data Retrieval and Storage</h3>
<hr/>
<p>
The purpose of this notebook is to present a simple but real world example of using Python to access remote data sources and perform Extract, Transform, and Load (ETL) processes to write this data into a relational database.  
</p>

In [1]:
# First we import the modules we will need
import pymysql.cursors  # Our MySQL connector
import pandas as pd # Our data manipulator - love this module
import configparser as cfp # A useful way to make multipurpose scripts that can be customized later
from datetime import datetime as dt # A module for creating date and time objects


In [2]:
# Now we will use our config parser and MySQL connector to open a connection to the database
con=cfp.ConfigParser()
con.read('/home/ryan/Python_Scripts/NDBC/config.ini') # This is a nifty way to store 
                                                      # connection parameters
my_host=con['LOCALDB']['host']                                    
my_port=int(con['LOCALDB']['port'])
my_user=con['LOCALDB']['user']
my_passwd=con['LOCALDB']['passwd']
my_db=con['LOCALDB']['db']
conn=pymysql.connect(host=my_host,port=my_port,user=my_user,passwd=my_passwd,
                   db=my_db)

In [3]:
# Now we build out some of the useful functions I know we will need.  It is not uncommon
# to wind up building these parts later as you build out your code but since we need these
# functions defined before we call them we will put them first.

# Building in fucntionality to replace old names with the correct ones (as of 2014)
def NDBCNames(oldname):
    namedict={  #old    #new
                'YYYY':'YY',
                '#YY':'YY',
                'WD':'WDIR',
                'DIR':'WDIR',
                'SPD':'WSPD',
                'GSP':'GST',
                'GNM':'GTIME',
                'BAR':'PRES',
                'BARO':'PRES',
                'H0':'WVHT',
                'DOMPD':'DPD',
                'AVP':'APD',
                'SRAD':'SWRAD',
                'SRAD2':'SWRAD',
                'LRAD':'LWRAD',
                'LRAD1':'LWRAD'
                }
    if oldname in namedict:
        return namedict[oldname]
    else:
        return oldname

In [4]:
# A function for getting the standard meteorological data for NDBC weather buoys
# It accepts a database cursor, station id, year and/or month for which data is desired
def getData(cursor,station,year,month):
    base_url = 'http://www.ndbc.noaa.gov/view_text_file.php?filename={}'.format(station)
    if month==0:
        target=base_url+'h'+str(year)+'.txt.gz&dir=data/historical/stdmet/'
    elif year==0:
        tm=dt(this_year,month,1)
        target=base_url+str(month)+str(this_year)+'.txt.gz&dir=data/stdmet/'+tm.strftime('%b')+'/'
    else:
        print('Not sure what you meant to do here but this is weird')
    # okay now that we have built our url that should point toward the text data file
    # let's grad that data file
        
    # we are trainwrecking in 2007 when (at least for station 46042)
    # NOAA started adding a second line to the header with the units
    # for each measurement.  It would be nice to capture this somewhere
    # in our DB but obviously it cannot be part of the numerical data.
    # Perhaps an additional table?
    try:
        my_d=pd.read_csv(target,header=0, engine='python',sep="\s+")
    except:
        if month==0:
            print("Year " +  str(year) + " is not available")
        elif year==0:
            print("Month " + str(month) + " is not available")
        return pd.DataFrame() # in order to properly evaluate the result we need to return an empty data frame...kind of wasteful memory wise but it works
    # Now let's get those pesky year, month, day, and
    # hour columns into a Timestamp
    # first we get the columns with our datetime info
    if my_d.iloc[0][0]=="#yr":
        my_d=insertUnits(cursor,my_d)
    cols=list(my_d.columns)
    if 'mm' in cols:
        cols=cols[0:cols.index('mm')+1]
        my_d.set_index(keys=cols,inplace=True)
    else:
        cols=cols[0:cols.index('hh')+1]
        my_d.set_index(keys=cols,inplace=True)
    my_dt=my_d.index.values
    dtime=[]
    for i in my_dt:
        if len(str(i[0]))==2:
            Y=i[0]+1900
        else:
            Y=i[0]
        if len(cols)==5:
            strtime=dt(int(Y),int(i[1]),int(i[2]),int(i[3]),int(i[4])).isoformat()
        else:
            strtime=dt(Y,i[1],i[2],i[3]).isoformat()
        dtime.append(strtime)
    my_d.index=dtime
    # Huzzah, we have our datetime strings as our index!  Since
    # it is ISO 8601 compliant this should be easily loaded into
    # a MySQL DB.  Now let's handle those bad data flags
    cols=list(my_d.columns) # we need to redo this to account for indexing
    # AFTER CAREFUL CONSIDERATION IT APPEARS TO ME THAT DEALING WITH BAD DATA
    # FLAGS WOULD BE EASIER DONE ONCE THE DATA HAS BEEN LOADED INTO THE DATABASE
    # THEREFORE I WILL SKIP THIS SECTION AND IMPLEMENT A BAD DATA FLAG PROTOCOL
    # LATER ON, AFTER INSERTING THE RECORDS.
    newcols=[];
    for col in cols:
        newcols.append(NDBCNames(col))

    my_d.columns=newcols
    my_d.fillna(999.0, inplace=True)
    return my_d


In [5]:
def insertStdMet(station,cursor,data):
    # This function takes in a station id, database cursor and an array of data.  At present
    # it assumes the data is a pandas dataaframe with the datetime value as the index
    # It may eventually be modified to be more flexible.  With the parameters
    # passed in, it goes row by row and builds an INSERT INTO SQL statement
    # that assumes each row in the data array represents a new record to be
    # added.
    fields=list(data.columns) # if our table has been constructed properly, these column names should map to the fields in the data table
    # Building the SQL string
    strSQL1='REPLACE INTO std_met (station_id,date_time,'
    strSQL2='VALUES (%s,%s,'
    for f in fields:
        strSQL1+=f+','
        strSQL2+='%s,'
    # trimming the last comma
    strSQL1=strSQL1[:-1]
    strSQL2=strSQL2[:-1]
    strSQL1+=") " + strSQL2 + ")"
    # Okay, now we have our SQL string.  Now we need to build the list of tuples
    # that will be passed along with it to the .executemany() function.
    tuplist=[]
    for i in range(len(data)):
        r=data.iloc[i][:]
        datatup=(station,r.name)
        for f in r:
            datatup+=(f,)
        tuplist.append(datatup)
    cursor.executemany(strSQL1,tuplist)
    conn.commit()

In [6]:
def insertUnits(cursor,data):
    # This function is designed to check whether or not the first line of the 
    # pandas data frame passed in contains additional string information, usually
    # the units for the parameters measured.  If so it will insert those units
    # into the units table in our database
    r=data.iloc[0][:]
    if isinstance(r[0],str): # if we have a string we assume r represents the units
        cols=list(data.columns) # getting our list of columns
        strSQL="REPLACE INTO units (parameter, unit) VALUES (%s,%s)"
        tuplist=[]
        if 'mm' in cols:
            start_index=cols.index('mm')+1
        else:
            start_index=cols.index('hh')+1
        for col in cols[start_index:]:
            datatup=(col,r[col])
            tuplist.append(datatup)
        cursor.executemany(strSQL,tuplist)
        data = data.iloc[1:][:]
    return data



In [11]:
with conn.cursor() as cur:
    conn.commit()
    sql= "SELECT station_id FROM ndbc_stations"
    cur.execute(sql)sql="REPLACE INTO `ndbc_stations` (`station_id`,) VALUES(%s)"
    cur.execute(sql,('46042'))
    
    stations=cur.fetchall()
    for station in stations:
        for id in station:
            for y in range(2000,2017):
                stdmet=getData(cur,id,y,0)
                if not stdmet.empty:
                    insertStdMet(station,cur,stdmet)
    conn.commit()

Year 2000 is not available
Year 2001 is not available
Year 2002 is not available
Year 2003 is not available
Year 2004 is not available
Year 2005 is not available
Year 2006 is not available
Year 2007 is not available
Year 2008 is not available
Year 2009 is not available
Year 2010 is not available
Year 2011 is not available
Year 2012 is not available
Year 2013 is not available
Year 2014 is not available
Year 2015 is not available
Year 2016 is not available


In [8]:
cur=conn.cursor()
cur.execute('SELECT `station_id` FROM ndbc_stations')
stations=cur.fetchall()

In [9]:
stations

(('46042',),)

In [10]:
for station in stations:
    for id in station:
        print(id)

46042


In [12]:
stdmet=getData('46042',cur,2015,0)

Year 2015 is not available


In [13]:
dt?

In [ ]:
dt.timedelta(hours=1)